In [1]:
from fasthtml.common import *
from fasthtml.jupyter import *
from IPython.display import display, HTML
from fasthtml.common import *
from fasthtml.jupyter import *

def create_server(app,rt):
    if IN_NOTEBOOK:
        for port in range(8000,8030):   
            if 'server' in globals():
                print(f"Server already running on port {globals()['server'].port} - stopping it")
                globals()['server'].stop()
            if is_port_free(port):
                server = JupyUvi(app, port=port)

                def HShow(comp, app):
                    @app.get('/')
                    def get(): return comp
                    display(HTML(f'<a href="http://localhost:{port}/" target="_blank">Open in new tab</a>'))
                    return HTMX("/",port=port)
                print(f"Starter server on port {port}")
                Show = partial(HShow, app=app)
                return app, rt, server, HShow, Show

# vue_src = Link(href='https://unpkg.com/vue@3/dist/vue.esm-browser.js', rel="modulepreload")

# mod_map = Script("""{
#     "imports": {
#       "vue": "https://unpkg.com/vue@3/dist/vue.esm-browser.js"
#     }
#   }""", type="importmap")

vue_global = Script(src="https://unpkg.com/vue@3/dist/vue.global.js")

app, rt = fast_app(hdrs=[vue_global])
app, rt, server, HShow, Show = create_server(app, rt)

Starter server on port 8000


In [2]:
def VueButton1():
    id = unqid()
    script=Script("""
    const { createApp } = Vue;
    
    createApp({data() { return{ count: 0 } }}).mount('#%s');
    """ % id, type="module")
    return (Div(Button("Count: {{ count }}", **{"@vue:mounted": "console.timeEnd('mount1')"}, _at_click="count++", v_bind="count"), id=id), script)

def VueButton2():
    id = unqid()
    script=Script("""
    Vue.createApp({data() { return{ count: 0 } }}).mount('#%s');
    """ % id)
    return (Div(Button("Count: {{ count }}", **{"@vue:mounted": "console.timeEnd('mount2')"}, _at_click="count++", v_bind="count"), id=id), script)

@rt
def swap1():
    return VueButton1()

@rt
def swap2():
    return VueButton2()

In [7]:
Show(
    Container(VueButton1(), Button("Test VBtn1", hx_get="/swap1", hx_swap="outerHTML", hx_target="this", onclick="console.time('mount1')"),
              Hr(style="margin: 0.5rem auto;"),
    Button("Test VBtn2", hx_get="/swap2", hx_swap="outerHTML", hx_target="this", onclick="console.time('mount2')"))
)

## Timing

Testing the time it takes to mount the Vue component using different import strategies. For each iteration, the notebook is fully reloaded so all cache & variables are reset upon timing.

---

Will begin testing the ES6 version of the CDN:

### No module preload/ header import/ importmap

#### Dynamic Import

Time to mount after HTMX swap: 

1. 51.24ms
2. 73.83ms
3. 57.74ms

#### ES6 Module Import

Time to mount after HTMX swap:

1. 15.29ms
2. 16.96ms
3. 15.74ms

### No module preload, using importmap in head

#### Dynamic Import

Time to mount after HTMX swap:

1. 58.21ms
2. 50.32ms
3. 60.62ms

#### ES6 Module Import

1. 16.07ms
2. 15.89ms
3. 10.84ms

### With module preload, No importmap

#### Dynamic Import

Time to mount after HTMX swap:

1. 26.12ms
2. 29.62ms
3. 20.88ms

#### ES6 Module Import

1. 15.81ms
2. 7.01ms
3. 9.86ms

### With module preload & importmap

#### Dynamic Import

Time to mount after HTMX swap:

1. 17.95ms
2. 23.69ms
3. 25.20ms

#### ES6 Module Import

1. 15.59ms
2. 6.77ms
3. 15.78ms

---

Testing the Global version of the CDN:

It is worth noting, subsequent mounts seem to become faster after the initial mounting of a vue component.

> Importmap and module preloading is not necessary, as the 'Vue' module is loaded into the globals object in the head during initial render

1. 15.16ms
2. 13.33ms
3. 13.21ms

## Findings

It seems not to matter which method you use, the standard Global CDN works perfectly without any apparent flickering, though this might change for larger scale applications.

If you prefer to keep javascript locality, opt for the module approach. A simple importmap option in the head is perfect without a module preload link, otherwise the globals options is perfectly fine.